# Mp 1 0 3_Process Selector
Illustrate the usage of the multiproc TSelector interfaces with the h1 analysis
example.




**Author:**  Anda Chelba, Gerardo Ganis  
<i><small>This notebook tutorial was automatically generated with <a href= "https://github.com/root-project/root/blob/master/documentation/doxygen/converttonotebook.py">ROOTBOOK-izer</a> from the macro found in the ROOT repository  on Thursday, August 29, 2019 at 02:16 AM.</small></i>

In [ ]:
%%cpp -d
#include "ROOT/RMakeUnique.hxx"
#include "TString.h"
#include "TROOT.h"
#include "TChain.h"
#include "TFileCollection.h"
#include "TH1F.h"
#include "TTreeReader.h"
#include "ROOT/TTreeProcessorMP.hxx"

const auto file0 = "http://root.cern.ch/files/h1/dstarmb.root";
const std::vector<std::string> files = {file0,
                                        "http://root.cern.ch/files/h1/dstarp1a.root",
                                        "http://root.cern.ch/files/h1/dstarp1b.root",
                                        "http://root.cern.ch/files/h1/dstarp2.root"};

Macosx may generate connection to windowserver errors

In [ ]:
gROOT->SetBatch(kTRUE);

TString selectorPath = gROOT->GetTutorialDir();
selectorPath += "/tree/h1analysis.C+";
std::cout << "selector used is: " << selectorPath << "\n";
auto sel = TSelector::GetSelector(selectorPath);

The following code generates a crash when davix is used for http
 Davix does not seem fork-safe; the problem has been reported to the
 Davix developers. For the time being we disable this part.
 To repoduce the problem, uncomment the next line.

 #define __reproduce_davix

In [ ]:
#if defined(__reproduce_davix)
auto fp = std::make_unique<TTree>(TFile::Open(file0));
auto tree = fp->Get<TTree>("h42");
#endif

ROOT::TTreeProcessorMP pool(3);

TList *out = nullptr;
#if defined(__reproduce_davix)

Ttreeprocessormp::process with a single tree

In [ ]:
out = pool.Process(*tree, *sel);
sel->GetOutputList()->Delete();
#endif

Ttreeprocessormp::process with single file name and tree name
 Note: we have less files than workers here

In [ ]:
out = pool.Process(file0, *sel, "h42");
sel->GetOutputList()->Delete();

Prepare datasets: vector of files, tfilecollection

In [ ]:
TChain ch;
TFileCollection fc;
for (auto &&file : files) {
   fc.Add(new TFileInfo(file.c_str()));
   ch.Add(file.c_str());
}

Ttreeprocessormp::process with vector of files and tree name
 Note: we have more files than workers here (different behaviour)

In [ ]:
out = pool.Process(files, *sel, "h42");
sel->GetOutputList()->Delete();

Ttreeprocessormp::process with tfilecollection, no tree name

In [ ]:
out = pool.Process(fc, *sel);
sel->GetOutputList()->Delete();

Ttreeprocessormp::process with tchain, no tree name

In [ ]:
out = pool.Process(ch, *sel);
sel->GetOutputList()->Delete();

return 0;